In [ ]:
import torch
from torch import nn
from torch.optim import Adam

class ClusteringNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_clusters):
        super(ClusteringNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_clusters)
        )
        self.centroids = nn.Parameter(torch.rand(n_clusters, hidden_dim))

    def forward(self, x):
        cluster_assignments = self.network(x)
        return cluster_assignments

def wcss_loss(outputs, centroids):
    """Calculate the Within-Cluster Sum of Squares (WCSS) loss."""
    norm_squared = torch.sum((outputs.unsqueeze(1) - centroids) ** 2, 2)
    min_norm_squared = torch.min(norm_squared, 1)[0]
    wcss = torch.sum(min_norm_squared)
    return wcss

def train_clustering_network(data_loader, model, optimizer, epochs):
    for epoch in range(epochs):
        total_loss = 0
        for data in data_loader:
            inputs = data[0]
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = wcss_loss(outputs, model.centroids)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data_loader)}')

# Example usage:
input_dim = 784  # Number of features in the input
hidden_dim = 64  # Size of the hidden layer
n_clusters = 10  # Number of clusters

In [ ]:
import torch
from torch import nn, optim
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from torch.utils.data import DataLoader, TensorDataset
# add tensor import

In [21]:
# Load the Iris dataset
iris = datasets.load_iris()
X = iris.data

# Standardize the dataset
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)

# Create a dataset and data loader
dataset = TensorDataset(X_tensor)
data_loader = DataLoader(dataset, batch_size=10, shuffle=True)

# Define the clustering neural network
class ClusteringNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_clusters):
        super(ClusteringNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_clusters),
            nn.Softmax(dim=1)
        )
        # Initialize centroids as a learnable parameter
        self.centroids = nn.Parameter(torch.rand(n_clusters, hidden_dim))

    def forward(self, x):
        # Apply the network and get raw scores for each cluster
        cluster_scores = self.network(x)
        return cluster_scores


# Initialize the network and optimizer
input_dim = 4  # Iris dataset has 4 features
hidden_dim = 10
n_clusters = 3  # Iris dataset has 3 classes
model = ClusteringNetwork(input_dim, hidden_dim, n_clusters)
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Train the network
def train(model, data_loader, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs in data_loader:
            features = inputs[0]  # Get the input features
            optimizer.zero_grad()
            outputs = model(features)
            print(outputs.shape)
            # loss = wcss_loss(outputs, model.centroids)
            loss = torch.sum((outputs.unsqueeze(1) - model.centroids) ** 2, 2)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data_loader)}')
        
        

# Train the model
train(model, data_loader, optimizer, epochs=100)


torch.Size([10, 3])


In [ ]:
data_loader

In [ ]:



# Define the WCSS loss function
def wcss_loss(outputs, centroids):
    """Calculate the Within-Cluster Sum of Squares (WCSS) loss."""
    # Ensure outputs are 2D and centroids are correctly shaped for broadcasting
    outputs_2d = outputs if outputs.dim() == 2 else outputs.unsqueeze(0)
    
    # Calculate the distance from each point to each centroid
    distances = torch.cdist(outputs_2d, centroids, p=2)
    
    # Assign each point to the closest centroid and calculate WCSS
    min_distances = distances.min(dim=1)[0]
    wcss = min_distances.pow(2).sum()
    return wcss





# Evaluate the clustering performance using silhouette score
def evaluate_clustering(model, data_loader):
    model.eval()
    all_outputs = []
    with torch.no_grad():
        for inputs, in data_loader:
            inputs = inputs[0]  # Get the input features
            outputs = model(inputs)
            all_outputs.extend(outputs.numpy())
    
    # Infer cluster assignments
    cluster_assignments = torch.argmax(torch.tensor(all_outputs), dim=1).numpy()
    
    # Calculate silhouette score using inferred cluster assignments
    silhouette_avg = silhouette_score(all_outputs, cluster_assignments)
    print(f'Silhouette Score (unsupervised): {silhouette_avg}')

# Evaluate the model
evaluate_clustering(model, data_loader)
